In [ ]:
# MARKOV DECISION PROCESS USING DYNAMIC PROGRAMMING
trans_data = pd.read_csv(r'transitions_data.csv',header=None)
rwd_data = pd.read_csv('rewards.csv',header=None)
trans_data = trans_data.to_numpy()
rwd_data = rwd_data.to_numpy()

gamma = 0.9
transitions = {}
len_data = trans_data.shape[0]
td = trans_data
for i in range(1,len_data):   
  if (td[i][0] in transitions):
    if td[i][1] in transitions[td[i][0]]:
      transitions[td[i][0]][td[i][1]].append((float(td[i][3]),td[i][2]))
    else:
      transitions[td[i][0]][td[i][1]] = [(float(td[i][3]),td[i][2])]
  else:
    transitions[td[i][0]] = {td[i][1]:[(float(td[i][3]),td[i][2])]}


rewards = {}
rd = rwd_data
len_rewards = rd.shape[0]
for i in range(0,len_rewards):
  rewards[rd[i][0]] = float(rd[i][1]) if rd[i][1] != 'None' else np.nan

class MarkovDecisionProcess:
  def __init__(self, states=[], transition={}, reward={}, gamma=0.9):
    self.states = states
    self.transition = transition
    self.reward = reward
    self.gamma = gamma

  def Rwd(self, state):
    return self.reward[state]
  
  def Trans(self, state, action):
    return self.transition[state][action]

  def action(self, state):
    return self.transition[state].keys()

Transitions = transitions
Rewards = rewards
States = transitions.keys()

mdp = MarkovDecisionProcess(states = States, transition = Transitions, reward = Rewards)
epsilon = 0.2
def val_iteration():
  states = mdp.states
  actions = mdp.action
  Trans = mdp.Trans
  Rwd = mdp.Rwd
  V1 = {s: 0 for s in states}
  while True:
    V = V1.copy()
    delta = 0 

    for s in states:
      V1[s] = Rwd(s) + gamma * max([sum([p*V[s1] for (p,s1) in Trans(s,a)]) for a in actions(s)])
      delta = max(delta, abs(V1[s]-V[s]))

    if (delta < epsilon*(1-gamma)/gamma):
      return V

def best_policy(V):
  states = mdp.states
  actions = mdp.action
  pi_policy = {}
  for s in states:
    pi_policy[s] = max(actions(s), key=lambda a: expected_utility(a,s,V))
  return pi_policy

In [ ]:
# monte carlo 
def monte_carlo_e_soft(env, episodes=100, policy=None, epsilon=0.01):
    if not policy:
        policy = create_random_policy(env)  # Create an empty dictionary to store state action values    
    Q = create_state_action_dictionary(env, policy) # Empty dictionary for storing rewards for each state-action pair
    returns = {} # 3.
    
    for _ in range(episodes): # Looping through episodes
        G = 0 # Store cumulative reward in G (initialized at 0)
        episode = run_game(env=env, policy=policy, display=False) # Store state, action and value respectively 
        
        # for loop through reversed indices of episode array. 
        # The logic behind it being reversed is that the eventual reward would be at the end. 
        # So we have to go back from the last timestep to the first one propagating result from the future.
        
        for i in reversed(range(0, len(episode))):   
            s_t, a_t, r_t = episode[i] 
            state_action = (s_t, a_t)
            G += r_t # Increment total reward by reward on current timestep
            
            if not state_action in [(x[0], x[1]) for x in episode[0:i]]: # 
                if returns.get(state_action):
                    returns[state_action].append(G)
                else:
                    returns[state_action] = [G]   
                    
                Q[s_t][a_t] = sum(returns[state_action]) / len(returns[state_action]) # Average reward across episodes
                
                Q_list = list(map(lambda x: x[1], Q[s_t].items())) # Finding the action with maximum value
                indices = [i for i, x in enumerate(Q_list) if x == max(Q_list)]
                max_Q = random.choice(indices)
                
                A_star = max_Q # 14.
                
                for a in policy[s_t].items(): # Update action probability for s_t in policy
                    if a[0] == A_star:
                        policy[s_t][a[0]] = 1 - epsilon + (epsilon / abs(sum(policy[s_t].values())))
                    else:
                        policy[s_t][a[0]] = (epsilon / abs(sum(policy[s_t].values())))

    return policy
    
env = gym.make("FrozenLake8x8-v1")
policy = monte_carlo_e_soft(env,episodes = 5000)
test_policy(policy,env)

In [ ]:
# Q Learning
def init_q(s, a, type="ones"):
    """
    @param s the number of states
    @param a the number of actions
    @param type random, ones or zeros for the initialization
    """
    if type == "ones":
        return np.ones((s, a))
    elif type == "random":
        return np.random.random((s, a))
    elif type == "zeros":
        return np.zeros((s, a))

def qlearning(alpha, gamma, epsilon, episodes, max_steps, n_tests, render = False, test=False):
    """
    @param alpha learning rate
    @param gamma decay factor
    @param epsilon for exploration
    @param max_steps for max step in each episode
    @param n_tests number of test episodes
    """
    env = gym.make('Taxi-v3')
    n_states, n_actions = env.observation_space.n, env.action_space.n
    Q = init_q(n_states, n_actions, type="ones")
    timestep_reward = []
    for episode in range(episodes):
        print(f"Episode: {episode}")
        s = env.reset()
        a = epsilon_greedy(Q, epsilon, n_actions, s)
        t = 0
        total_reward = 0
        done = False
        while t < max_steps:
            if render:
                env.render()
            t += 1
            s_, reward, done, info = env.step(a)
            total_reward += reward
            a_ = np.argmax(Q[s_, :])
            if done:
                Q[s, a] += alpha * ( reward  - Q[s, a] )
            else:
                Q[s, a] += alpha * ( reward + (gamma * Q[s_, a_]) - Q[s, a] )
            s, a = s_, a_
            if done:
                if render:
                    print(f"This episode took {t} timesteps and reward: {total_reward}")
                timestep_reward.append(total_reward)
                break
    if render:
        print(f"Here are the Q values:\n{Q}\nTesting now:")
    if test:
        test_agent(Q, env, n_tests, n_actions)
    return timestep_reward


def test_agent(Q, env, n_tests, n_actions, delay=1):
    for test in range(n_tests):
        print(f"Test #{test}")
        s = env.reset()
        done = False
        epsilon = 0
        while True:
            time.sleep(delay)
            env.render()
            a = epsilon_greedy(Q, epsilon, n_actions, s, train=True)
            print(f"Chose action {a} for state {s}")
            s, reward, done, info = env.step(a)
            if done:
                if reward > 0:
                    print("Reached goal!")
                else:
                    print("Shit! dead x_x")
                time.sleep(3)
                break

